In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import ArrayType, StringType, StructType, StructField
from pyspark.ml.fpm import FPGrowth # ml기반 fpgrowth는 fit으로 학습
import pyspark.sql.functions as F
import pandas as pd
!pip install psutil
import psutil

In [3]:
# SparkSession 초기화
spark = SparkSession.builder \
    .appName("nutr_fpgrowth") \
    .getOrCreate()

# 0. 데이터 불러오고 확인하기

In [4]:
pandas_df = pd.read_csv("./data/survey_total.csv")
pandas_df["섭취품목"] = pd.DataFrame(pandas_df["섭취품목"].apply(lambda x : eval(x)))
df = spark.createDataFrame(pandas_df)

C:\Users\USER\anaconda3\envs\spark_env2\lib\site-packages\pyspark\sql\pandas\conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [5]:
print(f"전체 데이터 행의 개수 : {df.count()}")
df.printSchema()
df.show(5)

전체 데이터 행의 개수 : 9560
root
 |-- 성별: string (nullable = true)
 |-- 연령대: string (nullable = true)
 |-- 섭취품목: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 비타민 및 무기질: long (nullable = true)
 |-- 식이섬유: long (nullable = true)
 |-- 아미노산 및 단백질: long (nullable = true)
 |-- 지방산: long (nullable = true)
 |-- 엽록소: long (nullable = true)
 |-- 인삼류: long (nullable = true)
 |-- 페놀류: long (nullable = true)
 |-- 당 및 탄수화물: long (nullable = true)
 |-- 발효미생물류: long (nullable = true)
 |-- 신규기능성식품: long (nullable = true)

+----+------+--------------------------------+----------------+--------+------------------+------+------+------+------+--------------+------------+--------------+
|성별|연령대|                        섭취품목|비타민 및 무기질|식이섬유|아미노산 및 단백질|지방산|엽록소|인삼류|페놀류|당 및 탄수화물|발효미생물류|신규기능성식품|
+----+------+--------------------------------+----------------+--------+------------------+------+------+------+------+--------------+------------+--------------+
|여자| 50~59|[비타민 및 무기질, 발효미생물...|    

In [6]:
df.select("연령대").distinct().orderBy(F.desc("연령대")).show()
df.select("연령대").distinct().orderBy(F.desc("연령대")).count()

+------+
|연령대|
+------+
| 90~99|
| 80~89|
| 70~79|
| 60~69|
| 50~59|
| 40~49|
| 30~39|
| 20~29|
| 10~19|
|   0~9|
+------+



10

# 1. FP-Growth 모델링
- 성별 및 연령대별 섭취품목을 fp-growth로 학습시키기
- 총 20개의 모델이 나와야하고 딕셔너리에 저장
- Spark를 사용한 모델링의 핵심은 RDD에 있습니다! 중요★★★
- 분산처리 개념에 대해서는 꼭 이해하시고 가셔야 해요

In [7]:
from datetime import datetime

start_time = datetime.now()
print(f"started at : {start_time}")

models = {}  # 각 성별 및 연령대별 모델 저장을 위한 dict

genders = df.select("성별").distinct().rdd.flatMap(lambda x: x).collect()
age_groups = df.select("연령대").distinct().rdd.flatMap(lambda x: x).collect()
# df.select("컬럼명") 선택 -> discinct() 중복값 제거
# rdd : 분산환경에서 데이터 병렬처리 지원
# flatMap(lambda x: x) : rdd 평평하게 만들기
# collect() : rdd내의 모든 요소 로컬로 수집

# 모든 성별 및 연령대 조합에 대해 subset 데이터프레임 생성
for gender in genders:
    for age_group in age_groups:
        subset = df.filter((df["성별"] == gender) & (df["연령대"] == age_group))
        # 모델 생성 및 학습
        # - itemsCol : 학습시킬 데이터 열
        # - minSupport : 최소 지지도 / minConfidence : 최소 신뢰도
        fp_growth = FPGrowth(itemsCol="섭취품목", minSupport=0.001, minConfidence=0.001)
        model = fp_growth.fit(subset)
        # 학습된 모델을 딕셔너리에 저장
        # - key : 성별-연령대 조합
        # - value : fp-growth 모델
        models[(gender, age_group)] = model

end_time = datetime.now()
print(f"ended at : {end_time}")
print(f"total : {end_time - start_time}")

started at : 2023-11-02 13:01:16.462381
ended at : 2023-11-02 13:03:51.537942
total : 0:02:35.075561


### 모델 저장하기
- 딕셔너리의 각 모델을 별도의 파일로 저장하고 파일 경로를 딕셔너리의 값으로 업데이트
- 단일 파일로 저장했을 때 보다 관리는 어렵지만, 모델끼리 독립적이고 최적화 시 업데이트가 유연함. 단일보다 모델 크기도 작음

In [8]:
def save_models(models):
    for (gender, age_group), model in models.items():
        path = f"./models/{gender}_{age_group}.model"
        model.write().overwrite().save(path)
        
save_models(models)

[Error] 
Py4JJavaError: An error occurred while calling o1334.save 트러블 슈팅
: ExitCodeException exitCode=-1073741515:   
- Hadoop 라이브러리와 관련 된 에러임. Hadoop 라이브러리 또는 연관 native 라이브러리 로드에 문제
- Window 환경에서 PySpark를 사용할 때 발생하는 문제랍니다
- Hdoop 바이너리 다운로드와 환경설정을 마쳤으니 권한문제라 생각
  1. 관리자 권한으로 cmd 실행 후 mkdir C:\tmp\hive (있으면 생략 가능)
  2. C:\hadoop\bin\winutils.exe chmod -R 777 C:\tmp\hive 명령어 실행
  3. 만약 MSVCR100.dll이 없어 코드 실행을 진행할 수 없다는 오류가 떴으면
  4. 파일 구글링 후 다운 받는다음 C:\Windows\System32에 넣기! 재부팅 하시면 모델이 저장됩니다
  -  참고 링크 : https://blog.naver.com/PostView.naver?blogId=jxbjultd&logNo=222838269355&parentCategoryNo=&categoryNo=6&viewDate=&isShowPopularPosts=true&from=search.

In [9]:
# def load_models(keys):
#     loaded_models = {}
#     for key in keys:
#         gender, age_group = key
#         path = f"/path/to/save/{gender}_{age_group}.model"
#         loaded_model = FPGrowthModel.load(path)
#         loaded_models[key] = loaded_model
#     return loaded_models

# # 성별 및 연령대별 모델 키 목록 (예시)
# model_keys = [('male', '20s'), ('male', '30s')]

# # 모델 로드
# models = load_models(model_keys)

In [10]:
# from pyspark.ml.fpm import FPGrowthModel

# # 딕셔너리를 pickle로부터 불러오기
# with open("models_dict.pkl", "rb") as f:
#     models_dict = pickle.load(f)

# # 모델 경로를 사용하여 각 모델 객체를 불러오기
# for (gender, age_group), model_path in models_dict.items():
#     models_dict[(gender, age_group)] = FPGrowthModel.load(model_path)

In [11]:
models

{('여자',
  '50~59'): FPGrowthModel: uid=FPGrowth_f3547ab28077, numTrainingRecords=1390,
 ('여자',
  '20~29'): FPGrowthModel: uid=FPGrowth_426ccf448888, numTrainingRecords=425,
 ('여자',
  '60~69'): FPGrowthModel: uid=FPGrowth_53b73fbe626f, numTrainingRecords=1035,
 ('여자',
  '30~39'): FPGrowthModel: uid=FPGrowth_61fc07f6accf, numTrainingRecords=697,
 ('여자',
  '40~49'): FPGrowthModel: uid=FPGrowth_f450774b4edf, numTrainingRecords=974,
 ('여자',
  '70~79'): FPGrowthModel: uid=FPGrowth_648c16438455, numTrainingRecords=265,
 ('여자',
  '80~89'): FPGrowthModel: uid=FPGrowth_86418aa2e258, numTrainingRecords=11,
 ('여자',
  '0~9'): FPGrowthModel: uid=FPGrowth_f6400f178944, numTrainingRecords=111,
 ('여자',
  '90~99'): FPGrowthModel: uid=FPGrowth_57d5cb4c996c, numTrainingRecords=0,
 ('여자',
  '10~19'): FPGrowthModel: uid=FPGrowth_3ff5de92a87b, numTrainingRecords=302,
 ('남자',
  '50~59'): FPGrowthModel: uid=FPGrowth_d88d6c395ccd, numTrainingRecords=1067,
 ('남자',
  '20~29'): FPGrowthModel: uid=FPGrowth_02833a32

In [12]:
records_values = [int(str(model).split("numTrainingRecords=")[-1].split(",")[0]) for model in models.values()]

# 총 합계 계산
total_records = sum(records_values)
print(total_records)

9560


### => 아래는 모델링 함수

In [13]:
# def train_models(df, item_col, gender_col, age_group_col):
#     models = {}  # 각 성별 및 연령대별 모델 저장을 위한 dict

#     genders = df.select(gender_col).distinct().rdd.flatMap(lambda x: x).collect()
#     age_groups = df.select(age_group_col).distinct().rdd.flatMap(lambda x: x).collect()

#     for gender in genders:
#         for age_group in age_groups:
#             subset = df.filter((df[gender_col] == gender) & (df[age_group_col] == age_group))
            
#             fp_growth = FPGrowth(itemsCol=item_col, minSupport=0.01, minConfidence=0.01)
#             model = fp_growth.fit(subset)
            
#             models[(gender, age_group)] = model
    
#     return models

# # 함수 사용 예
# trained_models = train_models(df, "섭취품목", "성별", "연령대")

# 2. 모델링 결과 분석

In [14]:
for (gender, age_group), model in models.items():
    print(f"성별: {gender}, 연령대: {age_group}")
    rules = model.associationRules
    filtered_rules = rules.filter(rules["lift"] > 1).orderBy(F.desc("lift"))
    filtered_rules.show(3)

성별: 여자, 연령대: 50~59
+-------------------------------+----------+------------------+------------------+--------------------+
|                     antecedent|consequent|        confidence|              lift|             support|
+-------------------------------+----------+------------------+------------------+--------------------+
| [당 및 탄수화물, 지방산, 발...|  [엽록소]|               0.4|11.120000000000001|0.001438848920863...|
|[아미노산 및 단백질, 인삼류,...|  [엽록소]|               0.4|11.120000000000001|0.001438848920863...|
|[아미노산 및 단백질, 인삼류,...|  [엽록소]|0.3333333333333333| 9.266666666666666|0.001438848920863...|
+-------------------------------+----------+------------------+------------------+--------------------+
only showing top 3 rows

성별: 여자, 연령대: 20~29
+--------------------------------+----------+------------------+------------------+--------------------+
|                      antecedent|consequent|        confidence|              lift|             support|
+--------------------------------+------

# 3. 모델링 예측
- fpgrowth의 prediction 기준 찾아보기

### i) transform을 사용한 code1
- lift가 높은 값을 반환하게 하려고 했더니 lift 컬럼이 없다고 함.
- prediction.select("*").show()를 해도 아래 섭취품목, prediction만 확인할 수 있음.
- transform의 prediction은 associationRules의 consequent 집합인 것을 확인하였음
- tranform은 각 트랜잭션에 대한 모든 연관 규칙을 찾아줌. prediction 리스트 안의 항목 순서는 신뢰도가 높은 순서임. (prediction[0]은 가장 신뢰도가 높은 항목!)
- 세가지 지표(confidence, lift, support)를 확인할 수 있는 associationRules에서 필터링 후 예측하기로 함

In [15]:
### fp-growth 예측 함수

# 성별, 연령대, 아이템 입력시 예측 결과 반환함
# def predict(gender, age_group, items):
#     model = models.get((gender, age_group))
#     if not model:
#         return "모델을 찾을 수 없습니다."
        
#     # 예측을 위한 DataFrame 생성
#     prediction_df = spark.createDataFrame([(items,)], ["섭취품목"])
#     predictions = model.transform(prediction_df)
#     # 첫 번째 예측 결과만 반환
#     #return prediction.select(F.col("prediction")[0]).show()
    
#     return predictions.select("*").show()

In [16]:
# # mysql에서 불러온 값을 아래 변수에 입력해주면 됩니다.
# gender_input = '여자'
# age_group_input = '20~29'
# items_input = ['신규기능성식품', '비타민 및 무기질']

# print(predict(gender_input, age_group_input, items_input))

### ii) associationRules를 사용한 code2
- Python의 리스트를 PySpark의 'ArrayType'으로 변환
- DataFrame에 변환된 배열을 추가할 때 용이함
- filter 연산을 사용하여 특정 조건을 충족하는 행들만 선택

In [21]:
from pyspark.sql.functions import lit, array, col, desc


def predictNutr(gender, age_group, items):
    # 시작 전 메모리 사용량 확인
    before_memory = psutil.virtual_memory().used / (1024.0 ** 3)  # GB 단위
    start_time = datetime.now()
    
    # 성별연령대별 모델 가지고오기
    model = models.get((gender, age_group))
    if not model:
        return "모델을 찾을 수 없습니다."

    # 불러온 모델 기반 규칙 찾기
    rules = model.associationRules

    # 아이템 리스트를 ArrayType으로 변환
    items_array = array(*[lit(item) for item in items])

    relevant_rules = rules.filter(col("antecedent") == items_array)
    
    if relevant_rules.count() == 0:
        return "해당 아이템에 대한 규칙을 찾을 수 없습니다."
        
    # 향상도 필터링
    filtered_rules = relevant_rules.filter(col("lift") > 1).orderBy(desc("lift"))
    # 연관규칙 지표 출력
    metrics = filtered_rules.select("consequent", "confidence", "lift", "support")

    # 성능측정
    end_time = datetime.now()
    after_memory = psutil.virtual_memory().used / (1024.0 ** 3)  # GB 단위
    
    print(f"Memory used before: {before_memory:.2f} GB")
    print(f"Memory used after: {after_memory:.2f} GB")
    print(f"Memory used for this operation: {after_memory - before_memory:.2f} GB")
    print(f"started at : {start_time}")
    print(f"ended at : {end_time}")
    print(f"total : {end_time - start_time}")
    
    return metrics.show()



gender_input = '여자'
age_group_input = '20~29'
items_input = ['신규기능성식품', '비타민 및 무기질']

print(predictNutr(gender_input, age_group_input, items_input))

Memory used before: 8.70 GB
Memory used after: 8.71 GB
Memory used for this operation: 0.02 GB
started at : 2023-11-02 13:09:55.947222
ended at : 2023-11-02 13:09:56.105286
total : 0:00:00.158064
+--------------------+-------------------+------------------+--------------------+
|          consequent|         confidence|              lift|             support|
+--------------------+-------------------+------------------+--------------------+
|            [페놀류]|0.03571428571428571|2.5297619047619047|0.002352941176470588|
|            [지방산]|0.17857142857142858|1.4880952380952381|0.011764705882352941|
|[아미노산 및 단백질]|0.07142857142857142|1.3198757763975155|0.004705882352941176|
+--------------------+-------------------+------------------+--------------------+

None


### associationRules를 사용한 code3
- Python 리스트를 직접 DataFrame의 ArrayType 컬럼으로 변환
- 새롭게 생성된 독립적인 DataFrame이며
- 다른 DataFrame과 조인하거나 다른 연산 수행 가능
- 다만, join을 사용할 경우 연산 비용이 크기 때문에 효율성 저하
- 아래 경우에는 단 하나의 행만 포함한 DataFrame이기 때문에 딱히 문제가 되지는 않음

In [22]:
# from pyspark.sql.functions import col


# def predictNutr(gender, age_group, items):
#     # 시작 전 메모리 사용량 확인
#     before_memory = psutil.virtual_memory().used / (1024.0 ** 3)  # GB 단위
#     start_time = datetime.now()
    
#     # 성별연령대별 모델 가지고오기
#     model = models.get((gender, age_group))
#     if not model:
#         return "모델을 찾을 수 없습니다."

#     # 불러온 모델 기반 규칙 찾기
#     rules = model.associationRules

#     # 아이템 리스트를 DataFrame의 ArrayType 컬럼으로 변환
#     items_df = spark.createDataFrame([(items,)], ["items_array"])

#     # 아이템 ArrayType을 가진 DataFrame과 규칙을 조인
#     relevant_rules = rules.join(items_df, rules["antecedent"] == col("items_array"))

#     if relevant_rules.count() == 0:
#         return "해당 아이템에 대한 규칙을 찾을 수 없습니다."
    
#     # 향상도 필터링
#     filtered_rules = relevant_rules.filter(col("lift") > 1).orderBy(desc("lift"))
#     # 연관규칙 지표 출력
#     metrics = filtered_rules.select("consequent", "confidence", "lift", "support")

#     #성능측정
#     end_time = datetime.now()
#     after_memory = psutil.virtual_memory().used / (1024.0 ** 3)  # GB 단위
    
#     print(f"Memory used before: {before_memory:.2f} GB")
#     print(f"Memory used after: {after_memory:.2f} GB")
#     print(f"Memory used for this operation: {after_memory - before_memory:.2f} GB")
#     print(f"started at : {start_time}")
#     print(f"ended at : {end_time}")
#     print(f"total : {end_time - start_time}")
    
#     return metrics.show()

# gender_input = '여자'
# age_group_input = '20~29'
# items_input = ['신규기능성식품', '비타민 및 무기질']

# print(predictNutr(gender_input, age_group_input, items_input))

Memory used before: 8.70 GB
Memory used after: 8.69 GB
Memory used for this operation: -0.01 GB
started at : 2023-11-02 13:10:05.811089
ended at : 2023-11-02 13:10:12.342671
total : 0:00:06.531582
+--------------------+-------------------+------------------+--------------------+
|          consequent|         confidence|              lift|             support|
+--------------------+-------------------+------------------+--------------------+
|            [페놀류]|0.03571428571428571|2.5297619047619047|0.002352941176470588|
|            [지방산]|0.17857142857142858|1.4880952380952381|0.011764705882352941|
|[아미노산 및 단백질]|0.07142857142857142|1.3198757763975155|0.004705882352941176|
+--------------------+-------------------+------------------+--------------------+

None


### 최종 코드 code2
- code2와 3을 비교했을 때, 메모리 사용량에서 큰 차이가 나지 않지만 처리 시간이 code2가 훨씬 빠름
  (code2: 0초 / code3: 약 6~8초)
- 코드의 간결성과 가독성을 고려하였을 때, code2의 filter를 사용하는 방법이 직관적이고 단순함
- (code3의 join은 DataFrame을 생성하고 join해야 함. 위 두 코드에서는 별 차이가 없으나 알아두기)
- 결론적으로 예측을 할 때 작은 규모의 데이터만 처리할 것이기 때문에, filter를 사용하는 방법 선택